In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.ensemble import VotingClassifier

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
training_text = pd.read_csv("../input/cancer-diagnosis/training_text",sep="\|\|", header=None, skiprows=1, names=["ID","Text"])

In [ ]:
training_text.head(5)

In [ ]:
training_text.shape

In [ ]:
training_variants = pd.read_csv("../input/cancer-diagnosis/training_variants")

In [ ]:
training_variants.head(5)

In [ ]:
training_variants.shape

In [ ]:
training_merge = training_variants.merge(training_text,left_on="ID",right_on="ID")

In [ ]:
training_merge.head(5)
#Now we have the data

In [ ]:
training_merge.shape

In [ ]:
test_text = pd.read_csv("../input/cancertreatment/stage2_test_text.csv",sep="\|\|", header=None, skiprows=1, names=["ID","Text"])

In [ ]:
test_text.shape

In [ ]:
test_variants = pd.read_csv("../input/cancertreatment/stage2_test_variants.csv")

In [ ]:
test_variants.shape

In [ ]:
test_merge = test_variants.merge(test_text,left_on="ID",right_on="ID")

In [ ]:
# Let's understand the type of values present in each column of our dataframe 'train_merge' dataframe.
test_merge.info()

In [ ]:
training_merge.describe(include='all')

In [ ]:
training_merge.isnull().sum()

In [ ]:
training_merge.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Plot to see how classes are distributed
plt.figure(figsize=(6,6))
sns.countplot(x="Class", data=training_merge)
plt.ylabel('Frequency', fontsize=12)
plt.xlabel('Class Count', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of Classes", fontsize=15)
plt.show()

In [ ]:
training_merge["Text_num_words"] = training_merge["Text"].apply(lambda x: len(str(x).split()) )
training_merge["Text_num_chars"] = training_merge["Text"].apply(lambda x: len(str(x)) )

In [ ]:
plt.figure(figsize=(6, 6))
sns.distplot(training_merge.Text_num_words.values, bins=50, kde=False, color='red')
plt.xlabel('Number of words in text', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title("Frequency of number of words", fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(training_merge.Text_num_chars.values, bins=50, kde=False, color='brown')
plt.xlabel('Number of characters in text', fontsize=12)
plt.ylabel('log of Count', fontsize=12)
plt.title("Frequency of Number of characters", fontsize=15)
plt.show()

In [ ]:
train_genes = training_merge.groupby('Gene')['Gene'].count()

for i in [2, 5, 10, 20, 50, 100, 300]:
    print('Genes that appear less than {} times: {}%'.format(i, round((train_genes < i).mean() * 100, 2)))

In [ ]:
training_merge['Variation'].describe()

In [ ]:
training_merge['Gene'].describe()

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1000, height=800).generate(" ".join(training_merge[training_merge.Class == 2]['Text']))
plt.figure(figsize=(10, 10))
plt.imshow(cloud)
plt.axis('off')
ax = plt.axes()

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1000, height=800).generate(" ".join(training_merge[training_merge.Class == 3]['Text']))
plt.figure(figsize=(10, 10))
plt.imshow(cloud)
plt.axis('off')
ax = plt.axes()

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1000, height=800).generate(" ".join(training_merge[training_merge.Class == 4]['Text']))
plt.figure(figsize=(10, 10))
plt.imshow(cloud)
plt.axis('off')
ax = plt.axes()


In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1000, height=800).generate(" ".join(training_merge[training_merge.Class == 5]['Text']))
plt.figure(figsize=(10, 10))
plt.imshow(cloud)
plt.axis('off')
ax = plt.axes()

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1000, height=800).generate(" ".join(training_merge[training_merge.Class == 8]['Text']))
plt.figure(figsize=(10, 10))
plt.imshow(cloud)
plt.axis('off')
ax = plt.axes()
ax.set_title('Class 2 Text Word Cloud')

In [ ]:
for i in range(1, 9):
    classs = training_merge.loc[training_merge['Class'] == i]

    cnt_genes = classs.Gene.value_counts().reset_index()
    cnt_genes.columns = ['Gene', 'Count']
    
    total = cnt_genes.Count.sum()
    
    cnt_genes['Percentage'] = (cnt_genes.Count/total)*100
    cnt_genes.plot(x='Gene', y='Percentage', kind='bar', figsize=(20,4))

In [ ]:
cnt_genes.head()

In [ ]:
cnt_genes.describe

In [ ]:
s = training_merge.Gene.value_counts().reset_index()
s.columns=['Gene', 'Count']

In [ ]:
cnt = s.Count.value_counts().reset_index()
cnt.columns = ['Occurence', 'Count']
total = cnt.Count.sum()

In [ ]:
s.loc[s['Count'] > 25].plot(x='Gene', y='Count', kind='bar', figsize=(16, 6))